In [1]:
#load the library
import numpy as np
import pandas as pd
import re
import seaborn as sns
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import optuna
import warnings
from tqdm import tqdm
import time
from sklearn.decomposition import TruncatedSVD, PCA
import joblib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
#transfrom categorical data to numbers
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [3]:
#import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('gender_submission.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## Fill NAs

In [7]:
sub.PassengerId.to_list() == test.PassengerId.to_list()

True

In [8]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
g1 = train[(train['Parch']==0)&(train['SibSp']==0)&(train['Pclass']==1)]['Age']
g2 = train[((train['Parch']!=0)|(train['SibSp']!=0))&(train['Pclass']==1)]['Age']
g3 = train[(train['Parch']==0)&(train['SibSp']==0)&(train['Pclass']==2)]['Age']
g4 = train[((train['Parch']!=0)|(train['SibSp']!=0))&(train['Pclass']==2)]['Age']
g5 = train[(train['Parch']==0)&(train['SibSp']==0)&(train['Pclass']==3)]['Age']
g6 = train[((train['Parch']!=0)|(train['SibSp']!=0))&(train['Pclass']==3)]['Age']
av1 = np.mean(g1)
av2 = np.mean(g2)
av3 = np.mean(g3)
av4 = np.mean(g4)
av5 = np.mean(g5)
av6 = np.mean(g6)

In [10]:
## fill age
train['Age'] = np.where(
    (train['Age'].isna()) & (train['Parch']==0) & (train['SibSp']==0) & (train['Pclass']==1), av1, train['Age'])

train['Age'] = np.where(
    (train['Age'].isna()) & (train['Parch']==0) & (train['SibSp']==0) & (train['Pclass']==2), av3, train['Age'])

train['Age'] = np.where(
    (train['Age'].isna()) & (train['Parch']==0) & (train['SibSp']==0) & (train['Pclass']==3), av5, train['Age'])

train['Age'] = np.where(
    (train['Age'].isna()) & ((train['Parch']!=0)|(train['SibSp']!=0))&(train['Pclass']==1), av2, train['Age'])

train['Age'] = np.where(
    (train['Age'].isna()) & ((train['Parch']!=0)|(train['SibSp']!=0))&(train['Pclass']==2), av4, train['Age'])

train['Age'] = np.where(
    (train['Age'].isna()) & ((train['Parch']!=0)|(train['SibSp']!=0))&(train['Pclass']==3), av6, train['Age'])



In [11]:
train.Cabin = train.Cabin.fillna('Unknown')

In [12]:
train[(train['Cabin'].str[0]=='B')]['Embarked'].value_counts()

S    23
C    22
Name: Embarked, dtype: int64

In [13]:
train.Embarked = train.Embarked.fillna('Unknown')

In [14]:
train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [15]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [16]:
# fill test age
g1 = test[(test['Parch']==0)&(test['SibSp']==0)&(test['Pclass']==1)]['Age']
g2 = test[((test['Parch']!=0)|(test['SibSp']!=0))&(test['Pclass']==1)]['Age']
g3 = test[(test['Parch']==0)&(test['SibSp']==0)&(test['Pclass']==2)]['Age']
g4 = test[((test['Parch']!=0)|(test['SibSp']!=0))&(test['Pclass']==2)]['Age']
g5 = test[(test['Parch']==0)&(test['SibSp']==0)&(test['Pclass']==3)]['Age']
g6 = test[((test['Parch']!=0)|(test['SibSp']!=0))&(test['Pclass']==3)]['Age']
av1 = np.mean(g1)
av2 = np.mean(g2)
av3 = np.mean(g3)
av4 = np.mean(g4)
av5 = np.mean(g5)
av6 = np.mean(g6)

test['Age'] = np.where(
    (test['Age'].isna()) & (test['Parch']==0) & (test['SibSp']==0) & (test['Pclass']==1), av1, test['Age'])

test['Age'] = np.where(
    (test['Age'].isna()) & (test['Parch']==0) & (test['SibSp']==0) & (test['Pclass']==2), av3, test['Age'])

test['Age'] = np.where(
    (test['Age'].isna()) & (test['Parch']==0) & (test['SibSp']==0) & (test['Pclass']==3), av5, test['Age'])

test['Age'] = np.where(
    (test['Age'].isna()) & ((test['Parch']!=0)|(test['SibSp']!=0))&(test['Pclass']==1), av2, test['Age'])

test['Age'] = np.where(
    (test['Age'].isna()) & ((test['Parch']!=0)|(test['SibSp']!=0))&(test['Pclass']==2), av4, test['Age'])

test['Age'] = np.where(
    (test['Age'].isna()) & ((test['Parch']!=0)|(test['SibSp']!=0))&(test['Pclass']==3), av6, test['Age'])



In [17]:
test.Cabin = test.Cabin.fillna('Unknown')

In [18]:
test.Fare = test.Fare.fillna(np.mean(test.Fare))

In [19]:
test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## Split folds

In [20]:
#do k fold validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
train['fold'] = -999
for fold_id, (train_index, valid_index) in enumerate(skf.split(X=train, y=train.Survived.values)):
    train.loc[valid_index, 'fold'] = fold_id

In [22]:
train.fold.value_counts()

0    179
1    178
2    178
3    178
4    178
Name: fold, dtype: int64

## Feature Engineering

#### 1. Name Title (Change the name to title and label encoding)

In [23]:
def find_title(entry):
    pattern = r'[a-zA-Z]+\.'
    return re.findall(pattern, entry)[0][:-1]

temp = train[['Name']]
temp['title'] =  temp['Name'].apply(find_title)
train = pd.merge(train, temp, on = 'Name')

In [24]:
train['title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Major         2
Mlle          2
Jonkheer      1
Ms            1
Countess      1
Mme           1
Lady          1
Capt          1
Sir           1
Don           1
Name: title, dtype: int64

In [25]:
def conversion_title(entry):
    if entry == 'Miss':
        return 'Miss'
    elif entry == 'Mrs':
        return 'Mrs'
    elif entry == 'Mr':
        return 'Mr'
    elif entry == 'Master':
        return 'Master'
    elif entry == 'Dr':
        return 'Dr'
    elif entry == 'Rev':
        return 'Rev'
    else:
        return 'ELSE'
train['title'] = train['title'].apply(conversion_title)

In [26]:
temp = test[['Name']]
temp['title'] = temp['Name'].apply(find_title)
test = pd.merge(test, temp, on = 'Name')
test['title'] = test['title'].apply(conversion_title)

In [27]:
train = train.drop('Name', axis=1)
test = test.drop('Name', axis=1)

In [28]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,Unknown,S,1,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,4,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,3,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,3,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,Unknown,S,0,Mr


In [29]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,892,3,male,34.5,0,0,330911,7.8292,Unknown,Q,Mr
1,893,3,female,47.0,1,0,363272,7.0000,Unknown,S,Mrs
2,894,2,male,62.0,0,0,240276,9.6875,Unknown,Q,Mr
3,895,3,male,27.0,0,0,315154,8.6625,Unknown,S,Mr
4,896,3,female,22.0,1,1,3101298,12.2875,Unknown,S,Mrs


#### 2.  Cut the Age and Fare column to different bins 

In [30]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,fold
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.514790,0.523008,0.381594,32.204208,1.997755
std,257.353842,0.486592,0.836071,13.309636,1.102743,0.806057,49.693429,1.415800
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,21.000000,0.000000,0.000000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,28.235556,0.000000,0.000000,14.454200,2.000000
75%,668.500000,1.000000,3.000000,36.000000,1.000000,0.000000,31.000000,3.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,4.000000


In [31]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,29.471072,0.447368,0.392344,35.627188
std,120.810458,0.841838,12.949363,0.896760,0.981429,55.840500
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.250000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,26.121053,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,36.375000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [32]:
train['Fare'] = pd.qcut(train['Fare'], 10)
test['Fare'] = pd.qcut(test['Fare'], 10)
train['Age'] = pd.qcut(train['Age'], 10)
test['Age'] = pd.qcut(test['Age'], 10)

In [33]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,title
0,1,0,3,male,"(19.784, 22.0]",1,0,A/5 21171,"(-0.001, 7.55]",Unknown,S,1,Mr
1,2,1,1,female,"(34.0, 40.0]",1,0,PC 17599,"(39.688, 77.958]",C85,C,4,Mrs
2,3,1,3,female,"(22.0, 27.0]",0,0,STON/O2. 3101282,"(7.854, 8.05]",Unknown,S,3,Miss
3,4,1,1,female,"(34.0, 40.0]",1,0,113803,"(39.688, 77.958]",C123,S,3,Mrs
4,5,0,3,male,"(34.0, 40.0]",0,0,373450,"(7.854, 8.05]",Unknown,S,0,Mr


In [34]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,892,3,male,"(33.0, 39.429]",0,0,330911,"(7.796, 8.05]",Unknown,Q,Mr
1,893,3,female,"(39.429, 48.0]",1,0,363272,"(-0.001, 7.644]",Unknown,S,Mrs
2,894,2,male,"(48.0, 76.0]",0,0,240276,"(8.05, 11.342]",Unknown,Q,Mr
3,895,3,male,"(26.121, 29.0]",0,0,315154,"(8.05, 11.342]",Unknown,S,Mr
4,896,3,female,"(20.129, 23.0]",1,1,3101298,"(11.342, 14.454]",Unknown,S,Mrs


### 3. clean ticket 

In [35]:
# Ticket  value_counts first
def ticket_extract(entry):
    pattern = r'[A-Z]+'
    extracted = ''.join(re.findall(pattern, entry))
    if extracted == '':
        return 'Number'
    elif extracted == 'PC':
        return extracted
    elif extracted == 'CA':
        return extracted
    else:
        return 'ELSE'

train['Ticket'] = train['Ticket'].apply(ticket_extract)
test['Ticket'] = test['Ticket'].apply(ticket_extract)

### 4. clean number of family members 

In [36]:
#train['Alone'] = np.where((train['SibSp']==0)&(train['Parch']==0), 1, 0)
#test['Alone'] = np.where((test['SibSp']==0)&(test['Parch']==0), 1, 0)

In [37]:
train['Family_size'] = np.zeros(len(train))

train['Family_size'] = np.where((train['SibSp']+train['Parch'])<3, 0, train['Family_size'])
train['Family_size'] = np.where((train['SibSp']+train['Parch']<5) & (train['SibSp']+train['Parch']>=3), 1, train['Family_size'])
train['Family_size'] = np.where((train['SibSp']+train['Parch'])>=5, 2, train['Family_size'])

test['Family_size'] = np.zeros(len(test))
test['Family_size'] = np.where((test['SibSp']+test['Parch'])<3, 0, test['Family_size'])
test['Family_size'] = np.where((test['SibSp']+test['Parch']<5) & (test['SibSp']+test['Parch']>=3), 1, test['Family_size'])
test['Family_size'] = np.where((test['SibSp']+test['Parch'])>=5, 2, test['Family_size'])

In [38]:
train = train.drop(columns = ['SibSp', 'Parch'])
test = test.drop(columns = ['SibSp', 'Parch'])

In [39]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,fold,title,Family_size
0,1,0,3,male,"(19.784, 22.0]",ELSE,"(-0.001, 7.55]",Unknown,S,1,Mr,0.0
1,2,1,1,female,"(34.0, 40.0]",PC,"(39.688, 77.958]",C85,C,4,Mrs,0.0
2,3,1,3,female,"(22.0, 27.0]",ELSE,"(7.854, 8.05]",Unknown,S,3,Miss,0.0
3,4,1,1,female,"(34.0, 40.0]",Number,"(39.688, 77.958]",C123,S,3,Mrs,0.0
4,5,0,3,male,"(34.0, 40.0]",Number,"(7.854, 8.05]",Unknown,S,0,Mr,0.0


### 6. Cabin number

In [40]:
train['Cabin Num'] = train['Cabin'].str.extract(r'([0-9]+)').fillna(0).astype(float)
test['Cabin Num'] = test['Cabin'].str.extract(r'([0-9]+)').fillna(0).astype(float)

In [41]:
train['Cabin Num'] = (train['Cabin Num']-np.mean(train['Cabin Num']))/np.std(train['Cabin Num'])
test['Cabin Num'] = (test['Cabin Num']-np.mean(test['Cabin Num']))/np.std(test['Cabin Num'])


### 5. Extract cabin value

In [42]:
#cabin  get first letter
def cabin(entry):
    rare_list = ['A', 'F', 'G', 'T']
    if entry=='Unknown':
        return 'NAN'
    elif entry[0] in rare_list:
        return 'Rare'
    else:
         return entry[0]
train['Cabin'] = train['Cabin'].apply(cabin)
test['Cabin'] = test['Cabin'].apply(cabin)

## Label Encoding

In [43]:
train = train.drop('PassengerId', axis=1)
test = test.drop('PassengerId', axis=1)

In [44]:
train.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,fold,title,Family_size,Cabin Num
0,0,3,male,"(19.784, 22.0]",ELSE,"(-0.001, 7.55]",NAN,S,1,Mr,0.0,-0.421322
1,1,1,female,"(34.0, 40.0]",PC,"(39.688, 77.958]",C,C,4,Mrs,0.0,2.738590
2,1,3,female,"(22.0, 27.0]",ELSE,"(7.854, 8.05]",NAN,S,3,Miss,0.0,-0.421322
3,1,1,female,"(34.0, 40.0]",Number,"(39.688, 77.958]",C,S,3,Mrs,0.0,4.151256
4,0,3,male,"(34.0, 40.0]",Number,"(7.854, 8.05]",NAN,S,0,Mr,0.0,-0.421322


In [45]:
#transfrom categorical data to numbers
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [46]:
train, test = label_encoding(train, test, ['Sex', 'Ticket', 'Embarked', 'Age', 'Cabin Num', 'Fare','Cabin', 'title'])

In [47]:
# train = pd.get_dummies(train)
# test= pd.get_dummies(test)

In [49]:
train.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,fold,title,Family_size,Cabin Num
0,0.0,3,1,4,1,0,4,2,1.0,4,0.0,0
1,1.0,1,0,15,3,15,1,0,4.0,5,0.0,113
2,1.0,3,0,6,1,5,4,2,3.0,3,0.0,0
3,1.0,1,0,15,2,15,1,2,3.0,5,0.0,142
4,0.0,3,1,15,2,5,4,2,0.0,4,0.0,0


In [50]:
test.head()

,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,fold,title,Family_size,Cabin Num
0,NaN,3,1,14,2,4,4,1,NaN,4,0.0,1
1,NaN,3,0,16,2,1,4,2,NaN,5,0.0,1
2,NaN,2,1,19,2,7,4,1,NaN,4,0.0,1
3,NaN,3,1,9,2,7,4,2,NaN,4,0.0,1
4,NaN,3,0,5,2,9,4,2,NaN,5,0.0,1


# Building Model

## LGB

In [51]:
def optuna_lgb(n_trials=500):
    
    def objective(trial):

        params = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-5, 10, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-5, 10, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 62),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 0.9),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 0.9),
            "bagging_freq": trial.suggest_int("bagging_freq", 2, 9),
            "min_child_samples": trial.suggest_int("min_child_samples", 33, 93),
            "max_depth": trial.suggest_int("max_depth", 2, 7)
        }

        # initialize oof 
        oof_train = np.zeros((len(train),))

        for i in range(5):
            train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
            train_y = train.query(f'fold!={i}').Survived

            valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
            valid_y = train.query(f'fold=={i}').Survived
            
            lgb_train = lgb.Dataset(train_x,
                                    train_y)

            lgb_eval = lgb.Dataset(valid_x,
                                   valid_y,
                                   reference=lgb_train)

            gbm = lgb.train(params,
                            lgb_train,
                            valid_sets=[lgb_train, lgb_eval],
                            num_boost_round=5000,
                            verbose_eval=-1, 
                            early_stopping_rounds=1000
                            )
            oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
            oof_train[valid_x.index] = gbm.predict(valid_x,
                                                   num_iteration=gbm.best_iteration)
            
            
        return accuracy_score(train.Survived, oof_train.round())
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best_params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }
    best_params.update(study.best_params)
    ## save trained model
    # initialize oof 
    oof_train = np.zeros((len(train),))
    print('*'*50)
    print('Exporting best models')
    print('*' * 50)
    # split folds using RSGKF
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        lgb_train = lgb.Dataset(train_x,
                                train_y)

        lgb_eval = lgb.Dataset(valid_x,
                               valid_y,
                               reference=lgb_train)

        gbm = lgb.train(best_params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_eval],
                        num_boost_round=50000,
                        verbose_eval=-1, 
                        early_stopping_rounds=1001
                        )

        oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        oof_train[valid_x.index] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)

        joblib.dump(gbm, f'lgb_fold{i}.pkl')

In [52]:
optuna_lgb()

[I 2021-10-03 20:36:55,064] A new study created in memory with name: no-name-735d94ff-d248-4fc9-b0e8-8d6c028cd1f8


Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[215]	training's binary_logloss: 0.382849	valid_1's binary_logloss: 0.398435
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[686]	training's binary_logloss: 0.357085	valid_1's binary_logloss: 0.39168
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[129]	training's binary_logloss: 0.378594	valid_1's binary_logloss: 0.449733
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[882]	training's binary_logloss: 0.347537	valid_1's binary_logloss: 0.411517
Training until validation scores don't improve for 1000 rounds


[I 2021-10-03 20:36:56,541] Trial 0 finished with value: 0.8237934904601572 and parameters: {'lambda_l1': 0.036102186969584435, 'lambda_l2': 0.020966553706159657, 'num_leaves': 36, 'feature_fraction': 0.575207476366264, 'bagging_fraction': 0.6317338138419455, 'bagging_freq': 5, 'min_child_samples': 57, 'max_depth': 2}. Best is trial 0 with value: 0.8237934904601572.


Early stopping, best iteration is:
[1039]	training's binary_logloss: 0.34427	valid_1's binary_logloss: 0.39932
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[46]	training's binary_logloss: 0.451007	valid_1's binary_logloss: 0.441181
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[374]	training's binary_logloss: 0.424054	valid_1's binary_logloss: 0.433275
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[415]	training's binary_logloss: 0.411312	valid_1's binary_logloss: 0.458606
Training until validation scores don't improve for 1000 rounds


[I 2021-10-03 20:36:57,388] Trial 1 finished with value: 0.7968574635241302 and parameters: {'lambda_l1': 9.101211703099091, 'lambda_l2': 0.04318613577702268, 'num_leaves': 24, 'feature_fraction': 0.5723142926385284, 'bagging_fraction': 0.6890449831072479, 'bagging_freq': 7, 'min_child_samples': 60, 'max_depth': 7}. Best is trial 0 with value: 0.8237934904601572.


...

[I 2021-10-03 20:41:16,534] Trial 139 finished with value: 0.8260381593714927 and parameters: {'lambda_l1': 0.01469076949250204, 'lambda_l2': 0.0035143955076931675, 'num_leaves': 22, 'feature_fraction': 0.6460063971279358, 'bagging_fraction': 0.5291504696450479, 'bagging_freq': 8, 'min_child_samples': 41, 'max_depth': 4}. Best is trial 71 with value: 0.8428731762065096.


Early stopping, best iteration is:
[880]	training's binary_logloss: 0.287638	valid_1's binary_logloss: 0.377369
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[1323]	training's binary_logloss: 0.293165	valid_1's binary_logloss: 0.384951
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[216]	training's binary_logloss: 0.36053	valid_1's binary_logloss: 0.387229
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[225]	training's binary_logloss: 0.338244	valid_1's binary_logloss: 0.456691
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[756]	training's binary_logloss: 0.300874	valid_1's binary_logloss: 0.408131
Training until validation scores don't improve for 1000 rounds


[I 2021-10-03 20:41:20,127] Trial 140 finished with value: 0.8316498316498316 and parameters: {'lambda_l1': 0.007251407859900425, 'lambda_l2': 3.334296193101823, 'num_leaves': 17, 'feature_fraction': 0.8272302527899054, 'bagging_fraction': 0.509964980253373, 'bagging_freq': 9, 'min_child_samples': 43, 'max_depth': 5}. Best is trial 71 with value: 0.8428731762065096.


Early stopping, best iteration is:
[1692]	training's binary_logloss: 0.275384	valid_1's binary_logloss: 0.378681
Training until validation scores don't improve for 1000 rounds


KeyboardInterrupt: 

## XGBoost

In [ ]:
#optuna example
#https://github.com/optuna/optuna-examples/blob/main/xgboost/xgboost_integration.py
import xgboost as xgb
def objective(trial):
    param = {
            "verbosity": 0,
            "objective": "binary:logistic",  #binary:logistic
            "eval_metric": "auc",  #map
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        
        accuracy = []
        # Add a callback for pruning.
        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
        bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
        preds = bst.predict(dvalid)
        pred_labels = np.rint(preds)
        accuracy.append(sklearn.metrics.accuracy_score(valid_y, pred_labels))
        
        
        joblib.dump(bst, f'xgboost_fold{i}.pkl')
    return np.mean(accuracy)

    
if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

[6]	validation-auc:0.87801
[7]	validation-auc:0.87654
[8]	validation-auc:0.87179
[9]	validation-auc:0.87286
[0]	validation-auc:0.82487
[1]	validation-auc:0.85308
[2]	validation-auc:0.85054
[3]	validation-auc:0.84880
[4]	validation-auc:0.85107
[5]	validation-auc:0.84626
[6]	validation-auc:0.82995
[7]	validation-auc:0.82874
[8]	validation-auc:0.82707
[9]	validation-auc:0.83135
[0]	validation-auc:0.86350
[1]	validation-auc:0.86190
[2]	validation-auc:0.86150
[3]	validation-auc:0.86110
[4]	validation-auc:0.86143
[5]	validation-auc:0.86023
[6]	validation-auc:0.86805
[7]	validation-auc:0.86765
[8]	validation-auc:0.86704
[9]	validation-auc:0.86678
[0]	validation-auc:0.84823
[1]	validation-auc:0.84756
[2]	validation-auc:0.85627
[3]	validation-auc:0.85840
[4]	validation-auc:0.86132
[5]	validation-auc:0.86139
[6]	validation-auc:0.86252
[7]	validation-auc:0.86518
[8]	validation-auc:0.86637
[9]	validation-auc:0.86451
[I 2021-10-03 12:37:26,970] Trial 60 finished with value: 0.8146067415730337 and p

### Logistic

In [ ]:
#logistic
from sklearn import linear_model

#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'LogReg'
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    classifier_obj = linear_model.LogisticRegression(C=logreg_c)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'logistic_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

### Catboost

In [ ]:
#optuna catboost example
#https://github.com/optuna/optuna-examples/blob/main/catboost/catboost_simple.py
import catboost as cb
def objective(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
        
        #save the five fold accuracy
        accuracy = []
        
        gbm = cb.CatBoostClassifier(**param)
        gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
        
        #save the best model for each fold 
        joblib.dump(gbm, f'catboost_fold{i}.pkl')
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-10-03 13:08:24,691] A new study created in memory with name: no-name-15ab1b0f-08e1-47e8-b6be-f4e20cc0912e
[I 2021-10-03 13:08:31,783] Trial 0 finished with value: 0.8089887640449438 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.055062637218881695, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 3.6933611318320017}. Best is trial 0 with value: 0.8089887640449438.
[I 2021-10-03 13:08:37,234] Trial 1 finished with value: 0.8146067415730337 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.08407485346821005, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 1 with value: 0.8146067415730337.
[I 2021-10-03 13:08:39,011] Trial 2 finished with value: 0.8258426966292135 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.01250352878243508, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 9.190912206263489}. Best is trial 2

## Ensemble lgb and xgboost

In [ ]:
X_train = train.drop(['Survived', 'fold'], axis=1).values
y_train = train[['Survived']].values

In [ ]:
test2 = test.drop(['Survived', 'fold'], axis=1).values

,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,fold,Family_size,Cabin Num,title_Dr,title_ELSE,title_Master,title_Miss,title_Mr,title_Mrs,title_Rev
0,NaN,3,1,14,2,4,4,1,NaN,0.0,0,0,0,0,0,1,0,0
1,NaN,3,0,16,2,1,4,2,NaN,0.0,0,0,0,0,0,0,1,0
2,NaN,2,1,19,2,7,4,1,NaN,0.0,0,0,0,0,0,1,0,0
3,NaN,3,1,9,2,7,4,2,NaN,0.0,0,0,0,0,0,1,0,0
4,NaN,3,0,5,2,9,4,2,NaN,0.0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,3,1,8,1,4,4,2,NaN,0.0,0,0,0,0,0,1,0,0
414,NaN,1,0,14,3,18,1,0,NaN,0.0,90,0,1,0,0,0,0,0
415,NaN,3,1,14,1,1,4,2,NaN,0.0,0,0,0,0,0,1,0,0
416,NaN,3,1,8,2,4,4,2,NaN,0.0,0,0,0,0,0,1,0,0


In [ ]:
#get the mean of best fold in lgb
lgb_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./lgb_fold{i}.pkl')
    lgb_pred += model.predict(test2) / 5
    
lgb_pred = lgb_pred.round().astype(int)

In [ ]:
#get the mean of best fold in xgboost
xgboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./xgboost_fold{i}.pkl')
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)
    xgboost_pred += model.predict_proba(test2)[:,1] / 5
    
xgboost_pred = xgboost_pred.round().astype(int)

In [ ]:
#get the mean of best fold in logistic  0.8426966292134831 0.78947 selected nice
logistic_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./logistic_fold{i}.pkl')
    logistic_pred += model.predict(test2) / 5
    
logistic_pred = logistic_pred.round().astype(int)

In [ ]:
#get the mean of best fold in catboost  0.8426966292134831  nice  0.77990 selected
catboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./catboost_fold{i}.pkl')
    catboost_pred += model.predict(test2) / 5
    
catboost_pred = catboost_pred.round().astype(int)

In [ ]:
#combine
f_pro = (lgb_pred + xgboost_pred + catboost_pred + logistic_pred)/4    #predict probability [0, 1]


In [ ]:
pred = []
for i in f_pro:
    if i > 0.5:
        pred.append(1)
    else:
        pred.append(0)

In [ ]:
sub.Survived = pred

In [ ]:
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sub.to_csv('submission_lastly.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f45f37b-2499-4526-9eb4-651deeca3084' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>